In [1]:
!pip install numpy pandas tensorflow scikit-learn matplotlib


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
# Step 3: Load your data
data = pd.read_csv('Exp_Mn_Mw_Value.txt', sep='\t')
data.head()

,Run,Factor A,Factor B,Factor C,Factor D,Response 1 (Experimental),Response 2 (Experimental)
0,1,110,7,50,10,1127.19,1321.65
1,2,85,13,50,10,1024.97,1339.35
2,3,101,1,500,60,1950.00,2878.90
3,4,101,1,500,60,2223.17,2989.00
4,5,50,10,50,10,1845.60,2690.50


In [5]:
# Step 4: Prepare the data
# Replace these column names with your actual column names:
input_columns = ['factor A', 'factor B', 'factor C', 'factor D']  # Change these names
output_columns = ['Response 1 (Experimental)', 'Response 2 (Experimental)']           # Change these names

In [6]:
print(data.columns.tolist())

['Run', 'Factor A', 'Factor B', 'Factor C', 'Factor D', 'Response 1 (Experimental)', 'Response 2 (Experimental)']


In [7]:
# Step 5: Split data into inputs (X) and outputs (y)
X = data.iloc[:, 1:5].values   # columns 1–4: Factor A–D
y = data.iloc[:, 5:7].values   # columns 5–6: Responses Mn, Mw

print(f"Input shape (X): {X.shape}")
print(f"Output shape (y): {y.shape}")

Input shape (X): (25, 4)
Output shape (y): (25, 2)


In [8]:
from sklearn.model_selection import train_test_split
import numpy as np

# Set a random seed for reproducibility
SEED = 5

# Step 1: Split into train_val (65%) and test (35%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.35, shuffle=True, random_state=SEED
)

# Step 2: Split train_val into train (~42%) and val (~22%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.35, shuffle=True, random_state=SEED
)

# Show shapes of each split
print(f"Training data: {X_train.shape[0]} samples")
print(f"Validation data: {X_val.shape[0]} samples")
print(f"Testing data: {X_test.shape[0]} samples")

# Optional: sanity check on all shapes
print("\nDetailed shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val:   {X_val.shape},   y_val:   {y_val.shape}")
print(f"X_test:  {X_test.shape},  y_test:  {y_test.shape}")


Training data: 10 samples
Validation data: 6 samples
Testing data: 9 samples

Detailed shapes:
X_train: (10, 4), y_train: (10, 2)
X_val:   (6, 4),   y_val:   (6, 2)
X_test:  (9, 4),  y_test:  (9, 2)


In [9]:
# Step 7: Scale the data (normalize)
from sklearn.preprocessing import StandardScaler

# Scale X (inputs)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled   = scaler_X.transform(X_val)
X_test_scaled  = scaler_X.transform(X_test)

# Scale y (outputs)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)

# Sanity check (optional)
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)


X_train_scaled shape: (10, 4)
y_train_scaled shape: (10, 2)


In [10]:
# Step 8: Create a neural network with more layers, dropout, and L2 regularization
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Dropout

model = Sequential([
    layers.Dense(16, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-5),
                 input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.1),
    layers.Dense(8, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-5)),
    Dropout(0.1),
    layers.Dense(8, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-5)),
    Dropout(0.1),
    layers.Dense(2, activation='tanh')  # 2 outputs: Mn and Mw
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

print("Neural network model with extra layers created!")
model.summary()


Neural network model with extra layers created!


C:\Users\brian\PycharmProjects\PythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 306 (1.20 KB)

 Trainable params: 306 (1.20 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Step 9: Train the model
from tensorflow.keras.callbacks import EarlyStopping

# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae', 'mape']
)

# Set up early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=25,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=500,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.1387 - mae: 0.8887 - mape: 424.6519 - val_loss: 0.9073 - val_mae: 0.7460 - val_mape: 179.8048
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 1.0437 - mae: 0.8413 - mape: 501.8436 - val_loss: 0.9004 - val_mae: 0.7442 - val_mape: 179.7010
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.9584 - mae: 0.8031 - mape: 177.7425 - val_loss: 0.8925 - val_mae: 0.7421 - val_mape: 179.6347
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.2554 - mae: 0.9335 - mape: 359.9770 - val_loss: 0.8863 - val_mae: 0.7400 - val_mape: 179.1840
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.8161 - mae: 0.7263 - mape: 156.5361 - val_loss: 0.8801 - val_mae: 0.7383 - val_mape: 179.0983
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.9409 - mae: 0.7721 - mape: 308.8373 - val_loss: 0.8734 - val_mae: 0.7364 - val_mape: 178.8994
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.9277 - mae: 0.7847

In [12]:
# Step 10: Make predictions on scaled inputs
y_train_pred_scaled = model.predict(X_train_scaled)
y_val_pred_scaled   = model.predict(X_val_scaled)
y_test_pred_scaled  = model.predict(X_test_scaled)

# Inverse-transform predictions and true values back to original scale
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled)
y_val_pred   = scaler_y.inverse_transform(y_val_pred_scaled)
y_test_pred  = scaler_y.inverse_transform(y_test_pred_scaled)

y_train_true = scaler_y.inverse_transform(y_train_scaled)
y_val_true   = scaler_y.inverse_transform(y_val_scaled)
y_test_true  = scaler_y.inverse_transform(y_test_scaled)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [13]:
# Step 11: Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score

# Step 11: Evaluate the model
def evaluate_model(y_true, y_pred, dataset_name):
    """
    Calculate and print performance metrics for a 2-output regression model.
    y_true and y_pred must be 2D arrays: [:, 0] = Mn, [:, 1] = Mw
    """
    print(f"\n {dataset_name} Results:")
    
    # Mn
    mn_mse = mean_squared_error(y_true[:, 0], y_pred[:, 0])
    mn_r2  = r2_score(y_true[:, 0], y_pred[:, 0])
    print(f"Mn - MSE: {mn_mse:.4f}, R²: {mn_r2:.4f}")
    
    # Mw
    mw_mse = mean_squared_error(y_true[:, 1], y_pred[:, 1])
    mw_r2  = r2_score(y_true[:, 1], y_pred[:, 1])
    print(f"Mw - MSE: {mw_mse:.4f}, R²: {mw_r2:.4f}")


In [14]:
import pandas as pd

# Step 13: Create a DataFrame of actual vs predicted values and errors
results_df = pd.DataFrame({
    'Actual_Mn': y_test_true[:, 0],
    'Predicted_Mn': y_test_pred[:, 0],
    'Actual_Mw': y_test_true[:, 1],
    'Predicted_Mw': y_test_pred[:, 1]
})

# Compute absolute errors
results_df['Mn_Error'] = abs(results_df['Actual_Mn'] - results_df['Predicted_Mn'])
results_df['Mw_Error'] = abs(results_df['Actual_Mw'] - results_df['Predicted_Mw'])

# Display first 10 predictions
print("\n📄 First 10 predictions:")
display(results_df.head(10))



📄 First 10 predictions:


,Actual_Mn,Predicted_Mn,Actual_Mw,Predicted_Mw,Mn_Error,Mw_Error
0,4663.77,2751.494873,5921.61,3860.809082,1912.275127,2060.800918
1,4663.04,2751.494873,5921.49,3860.809082,1911.545127,2060.680918
2,1950.00,2952.078125,2878.90,4131.125977,1002.078125,1252.225977
3,2322.86,2355.532715,2987.28,2974.999756,32.672715,12.280244
4,1265.88,1967.965454,1458.13,2414.153076,702.085454,956.023076
5,3764.53,2752.531494,5752.15,3829.603271,1011.998506,1922.546729
6,2590.79,2826.128174,3517.86,3909.973877,235.338174,392.113877
7,2752.80,2370.553711,3129.61,3010.101562,382.246289,119.508438
8,2951.90,2742.733398,2965.54,3793.340576,209.166602,827.800576
